# Home Sales

In [1]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession
import time

import findspark
findspark.init()

# Data collection

In [2]:
spark = SparkSession.builder.appName("MSUDA-challenge-sparksql").getOrCreate()

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 12:09:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
home_sales_df.createOrReplaceTempView("home_sales")

In [4]:
# verification
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

# Queries

In [5]:
# What is the average price, rounded to 2 decimals, of a four-bedroom house for each year?
spark.sql(
    """
    SELECT round(AVG(price), 2) as price_average, YEAR(date) as year
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    
    ORDER BY year
    """
).show()

+-------------+----+
|price_average|year|
+-------------+----+
|     300263.7|2019|
|    298353.78|2020|
|    301819.44|2021|
|    296363.88|2022|
+-------------+----+



In [6]:
# What is the average price, rounded to 2 decimals, of a home with three bedrooms and three bathrooms for each year the home was built?
spark.sql(
    """
    SELECT ROUND(AVG(price), 2) as price_average, date_built as year_built
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year_built

    ORDER BY year_built
    """
).show()

+-------------+----------+
|price_average|year_built|
+-------------+----------+
|    292859.62|      2010|
|    291117.47|      2011|
|    293683.19|      2012|
|    295962.27|      2013|
|    290852.27|      2014|
|     288770.3|      2015|
|    290555.07|      2016|
|    292676.79|      2017|
+-------------+----------+



In [7]:
# What is the average price, rounded to 2 decimals, of a home with three bedrooms, three bathrooms, two floors, and is >=2000 square feet for each year the home was built?
spark.sql(
    """
    SELECT ROUND(AVG(price), 2) as price_average, date_built as year_built
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000
    GROUP BY year_built

    ORDER BY year_built
    """
).show()

+-------------+----------+
|price_average|year_built|
+-------------+----------+
|    280447.23|      2010|
|    281413.45|      2011|
|    295858.68|      2012|
|    295142.13|      2013|
|    294195.13|      2014|
|    291788.36|      2015|
|     287812.6|      2016|
|    282026.59|      2017|
+-------------+----------+



In [8]:
# What is the average price, rounded to 2 decimals, of a home with an average price >= $350,000 for each view rating
start_time = time.time()
spark.sql(
    """
    SELECT ROUND(AVG(price), 2) as price_average, view as view_rating
    FROM home_sales
    GROUP BY view_rating
    
    HAVING price_average >= 350000
    ORDER BY view_rating
    """
).show()
end_time = time.time()

print(f"Query executed in {round(end_time-start_time, 2)} seconds")

+-------------+-----------+
|price_average|view_rating|
+-------------+-----------+
|    788128.21|         51|
|    733780.26|         52|
|     755214.8|         53|
|    798684.82|         54|
|    771153.32|         55|
|     718176.4|         56|
|     734340.5|         57|
|    759764.65|         58|
|     791453.0|         59|
|    754939.65|         60|
|    746877.59|         61|
|    759150.14|         62|
|    711614.55|         63|
|    767036.67|         64|
|    736679.93|         65|
|     712475.0|         66|
|    737970.96|         67|
|    716785.44|         68|
|    750537.94|         69|
|    695865.58|         70|
+-------------+-----------+
only showing top 20 rows

Query executed in 0.2 seconds
